In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## FractalNet

In [ ]:
"""
    FractalNet for CIFAR, implemented in PyTorch.
    Original paper: 'FractalNet: Ultra-Deep Neural Networks without Residuals,' https://arxiv.org/abs/1605.07648.
"""

__all__ = ['CIFARFractalNet', 'fractalnet_cifar10', 'fractalnet_cifar100']

import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.init as init

class ParametricSequential(nn.Sequential):
    """
    A sequential container for modules with parameters.
    Modules will be executed in the order they are added.
    """
    def __init__(self, *args):
        super(ParametricSequential, self).__init__(*args)

    def forward(self, x, **kwargs):
        for module in self._modules.values():
            x = module(x, **kwargs)
        return x

class DropConvBlock(nn.Module):
    """
    Convolution block with Batch normalization, ReLU activation, and Dropout layer.

    Parameters:
    ----------
    in_channels : int
        Number of input channels.
    out_channels : int
        Number of output channels.
    kernel_size : int or tuple/list of 2 int
        Convolution window size.
    stride : int or tuple/list of 2 int
        Strides of the convolution.
    padding : int or tuple/list of 2 int
        Padding value for convolution layer.
    bias : bool, default False
        Whether the layer uses a bias vector.
    dropout_rate : float, default 0.0
        Parameter of Dropout layer. Faction of the input units to drop.
    """
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 stride,
                 padding,
                 bias=False,
                 dropout_prob=0.0):
        super(DropConvBlock, self).__init__()
        self.use_dropout = (dropout_prob != 0.0)

        self.conv = nn.Conv2d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
            bias=bias)
        self.bn = nn.BatchNorm2d(num_features=out_channels)
        self.activ = nn.ReLU(inplace=True)
        if self.use_dropout:
            self.dropout = nn.Dropout2d(p=dropout_prob)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.activ(x)
        if self.use_dropout:
            x = self.dropout(x)
        return x


def drop_conv3x3_block(in_channels,
                       out_channels,
                       stride=1,
                       padding=1,
                       bias=False,
                       dropout_prob=0.0):
    """
    3x3 version of the convolution block with dropout.

    Parameters:
    ----------
    in_channels : int
        Number of input channels.
    out_channels : int
        Number of output channels.
    stride : int or tuple/list of 2 int, default 1
        Strides of the convolution.
    padding : int or tuple/list of 2 int, default 1
        Padding value for convolution layer.
    bias : bool, default False
        Whether the layer uses a bias vector.
    dropout_rate : float, default 0.0
        Parameter of Dropout layer. Faction of the input units to drop.
    """
    return DropConvBlock(
        in_channels=in_channels,
        out_channels=out_channels,
        kernel_size=3,
        stride=stride,
        padding=padding,
        bias=bias,
        dropout_prob=dropout_prob)


class FractalBlock(nn.Module):
    """
    FractalNet block.

    Parameters:
    ----------
    in_channels : int
        Number of input channels.
    out_channels : int
        Number of output channels.
    num_columns : int
        Number of columns in each block.
    loc_drop_prob : float
        Local drop path probability.
    dropout_prob : float
        Probability of dropout.
    """
    def __init__(self,
                 in_channels,
                 out_channels,
                 num_columns,
                 loc_drop_prob,
                 dropout_prob):
        super(FractalBlock, self).__init__()
        assert (num_columns >= 1)
        self.num_columns = num_columns
        self.loc_drop_prob = loc_drop_prob

        self.blocks = nn.Sequential()
        depth = 2 ** (num_columns - 1)
        for i in range(depth):
            level_block_i = nn.Sequential()
            for j in range(self.num_columns):
                column_step_j = 2 ** j
                if (i + 1) % column_step_j == 0:
                    in_channels_ij = in_channels if (i + 1 == column_step_j) else out_channels
                    level_block_i.add_module("subblock{}".format(j + 1), drop_conv3x3_block(
                        in_channels=in_channels_ij,
                        out_channels=out_channels,
                        dropout_prob=dropout_prob))
            self.blocks.add_module("block{}".format(i + 1), level_block_i)

    @staticmethod
    def calc_drop_mask(batch_size,
                       glob_num_columns,
                       curr_num_columns,
                       max_num_columns,
                       loc_drop_prob):
        """
        Calculate drop path mask.

        Parameters:
        ----------
        batch_size : int
            Size of batch.
        glob_num_columns : int
            Number of columns in global drop path mask.
        curr_num_columns : int
            Number of active columns in the current level of block.
        max_num_columns : int
            Number of columns for all network.
        loc_drop_prob : float
            Local drop path probability.

        Returns
        -------
        Tensor
            Resulted mask.
        """
        glob_batch_size = glob_num_columns.shape[0]
        glob_drop_mask = np.zeros((curr_num_columns, glob_batch_size), dtype=np.float32)
        glob_drop_num_columns = glob_num_columns - (max_num_columns - curr_num_columns)
        glob_drop_indices = np.where(glob_drop_num_columns >= 0)[0]
        glob_drop_mask[glob_drop_num_columns[glob_drop_indices], glob_drop_indices] = 1.0

        loc_batch_size = batch_size - glob_batch_size
        loc_drop_mask = np.random.binomial(
            n=1,
            p=(1.0 - loc_drop_prob),
            size=(curr_num_columns, loc_batch_size)).astype(np.float32)
        alive_count = loc_drop_mask.sum(axis=0)
        dead_indices = np.where(alive_count == 0.0)[0]
        loc_drop_mask[np.random.randint(0, curr_num_columns, size=dead_indices.shape), dead_indices] = 1.0

        drop_mask = np.concatenate((glob_drop_mask, loc_drop_mask), axis=1)
        return torch.from_numpy(drop_mask)

    @staticmethod
    def join_outs(raw_outs,
                  glob_num_columns,
                  num_columns,
                  loc_drop_prob,
                  training):
        """
        Join outputs for current level of block.

        Parameters:
        ----------
        raw_outs : list of Tensor
            Current outputs from active columns.
        glob_num_columns : int
            Number of columns in global drop path mask.
        num_columns : int
            Number of columns for all network.
        loc_drop_prob : float
            Local drop path probability.
        training : bool
            Whether training mode for network.

        Returns
        -------
        Tensor
            Joined output.
        """
        curr_num_columns = len(raw_outs)
        out = torch.stack(raw_outs, dim=0)
        assert (out.size(0) == curr_num_columns)

        if training:
            batch_size = out.size(1)
            batch_mask = FractalBlock.calc_drop_mask(
                batch_size=batch_size,
                glob_num_columns=glob_num_columns,
                curr_num_columns=curr_num_columns,
                max_num_columns=num_columns,
                loc_drop_prob=loc_drop_prob)
            batch_mask = batch_mask.to(out.device)
            assert (batch_mask.size(0) == curr_num_columns)
            assert (batch_mask.size(1) == batch_size)
            batch_mask = batch_mask.unsqueeze(2).unsqueeze(3).unsqueeze(4)
            masked_out = out * batch_mask
            num_alive = batch_mask.sum(dim=0)
            num_alive[num_alive == 0.0] = 1.0
            out = masked_out.sum(dim=0) / num_alive
        else:
            out = out.mean(dim=0)

        return out

    def forward(self, x, glob_num_columns):
        outs = [x] * self.num_columns

        for level_block_i in self.blocks._modules.values():
            outs_i = []

            for j, block_ij in enumerate(level_block_i._modules.values()):
                input_i = outs[j]
                outs_i.append(block_ij(input_i))

            joined_out = FractalBlock.join_outs(
                raw_outs=outs_i[::-1],
                glob_num_columns=glob_num_columns,
                num_columns=self.num_columns,
                loc_drop_prob=self.loc_drop_prob,
                training=self.training)

            len_level_block_i = len(level_block_i._modules.values())
            for j in range(len_level_block_i):
                outs[j] = joined_out

        return outs[0]


class FractalUnit(nn.Module):
    """
    FractalNet unit.

    Parameters:
    ----------
    in_channels : int
        Number of input channels.
    out_channels : int
        Number of output channels.
    num_columns : int
        Number of columns in each block.
    loc_drop_prob : float
        Local drop path probability.
    dropout_prob : float
        Probability of dropout.
    """
    def __init__(self,
                 in_channels,
                 out_channels,
                 num_columns,
                 loc_drop_prob,
                 dropout_prob):
        super(FractalUnit, self).__init__()
        self.block = FractalBlock(
            in_channels=in_channels,
            out_channels=out_channels,
            num_columns=num_columns,
            loc_drop_prob=loc_drop_prob,
            dropout_prob=dropout_prob)
        self.pool = nn.MaxPool2d(
            kernel_size=2,
            stride=2)

    def forward(self, x, glob_num_columns):
        x = self.block(x, glob_num_columns=glob_num_columns)
        x = self.pool(x)
        return x


class CIFARFractalNet(nn.Module):
    """
    FractalNet model for CIFAR from 'FractalNet: Ultra-Deep Neural Networks without Residuals,'
    https://arxiv.org/abs/1605.07648.

    Parameters:
    ----------
    channels : list of int
        Number of output channels for each unit.
    num_columns : int
        Number of columns in each block.
    dropout_probs : list of float
        Probability of dropout in each block.
    loc_drop_prob : float
        Local drop path probability.
    glob_drop_ratio : float
        Global drop part fraction.
    in_channels : int, default 3
        Number of input channels.
    in_size : tuple of two ints, default (32, 32)
        Spatial size of the expected input image.
    num_classes : int, default 10
        Number of classification classes.
    """
    def __init__(self,
                 channels,
                 num_columns,
                 dropout_probs,
                 loc_drop_prob,
                 glob_drop_ratio,
                 in_channels=3,
                 in_size=(32, 32),
                 num_classes=10):
        super(CIFARFractalNet, self).__init__()
        self.in_size = in_size
        self.num_classes = num_classes
        self.glob_drop_ratio = glob_drop_ratio
        self.num_columns = num_columns

        self.features = ParametricSequential()
        for i, out_channels in enumerate(channels):
            dropout_prob = dropout_probs[i]
            self.features.add_module("unit{}".format(i + 1), FractalUnit(
                in_channels=in_channels,
                out_channels=out_channels,
                num_columns=num_columns,
                loc_drop_prob=loc_drop_prob,
                dropout_prob=dropout_prob))
            in_channels = out_channels

        self.output = nn.Linear(
            in_features=in_channels,
            out_features=num_classes)

        self._init_params()

    def _init_params(self):
        for name, module in self.named_modules():
            if isinstance(module, nn.Conv2d):
                init.kaiming_uniform_(module.weight)
                if module.bias is not None:
                    init.constant_(module.bias, 0)

    def forward(self, x):
        glob_batch_size = int(x.size(0) * self.glob_drop_ratio)
        glob_num_columns = np.random.randint(0, self.num_columns, size=(glob_batch_size,))

        x = self.features(x, glob_num_columns=glob_num_columns)
        x = x.view(x.size(0), -1)
        x = self.output(x)
        return x


def get_fractalnet_cifar(num_classes,
                         model_name=None,
                         pretrained=False,
                         root=os.path.join("~", ".torch", "models"),
                         **kwargs):
    """
    Create WRN model for CIFAR with specific parameters.

    Parameters:
    ----------
    num_classes : int
        Number of classification classes.
    model_name : str or None, default None
        Model name for loading pretrained model.
    pretrained : bool, default False
        Whether to load the pretrained weights for model.
    root : str, default '~/.torch/models'
        Location for keeping the model parameters.
    """

    dropout_probs = (0.0, 0.1, 0.2, 0.3, 0.4)
    channels = [64 * (2 ** (i if i != len(dropout_probs) - 1 else i - 1)) for i in range(len(dropout_probs))]
    num_columns = 3
    loc_drop_prob = 0.15
    glob_drop_ratio = 0.5

    net = CIFARFractalNet(
        channels=channels,
        num_columns=num_columns,
        dropout_probs=dropout_probs,
        loc_drop_prob=loc_drop_prob,
        glob_drop_ratio=glob_drop_ratio,
        num_classes=num_classes,
        **kwargs)

    if pretrained:
        if (model_name is None) or (not model_name):
            raise ValueError("Parameter `model_name` should be properly initialized for loading pretrained model.")
        from .model_store import download_model
        download_model(
            net=net,
            model_name=model_name,
            local_model_store_dir_path=root)

    return net


def fractalnet_cifar10(num_classes=10, **kwargs):
    """
    FractalNet model for CIFAR-10 from 'FractalNet: Ultra-Deep Neural Networks without Residuals,'
    https://arxiv.org/abs/1605.07648.

    Parameters:
    ----------
    num_classes : int, default 10
        Number of classification classes.
    pretrained : bool, default False
        Whether to load the pretrained weights for model.
    root : str, default '~/.torch/models'
        Location for keeping the model parameters.
    """
    return get_fractalnet_cifar(num_classes=num_classes, model_name="fractalnet_cifar10", **kwargs)


def fractalnet_cifar100(num_classes=100, **kwargs):
    """
    FractalNet model for CIFAR-100 from 'FractalNet: Ultra-Deep Neural Networks without Residuals,'
    https://arxiv.org/abs/1605.07648.

    Parameters:
    ----------
    num_classes : int, default 100
        Number of classification classes.
    pretrained : bool, default False
        Whether to load the pretrained weights for model.
    root : str, default '~/.torch/models'
        Location for keeping the model parameters.
    """
    return get_fractalnet_cifar(num_classes=num_classes, model_name="fractalnet_cifar100", **kwargs)


def _calc_width(net):
    import numpy as np
    net_params = filter(lambda p: p.requires_grad, net.parameters())
    weight_count = 0
    for param in net_params:
        weight_count += np.prod(param.size())
    return weight_count


def _test():
    import torch

    pretrained = False

    models = [
        (fractalnet_cifar10, 10),
        (fractalnet_cifar100, 100),
    ]

    for model, num_classes in models:

        net = model(pretrained=pretrained)

        # net.train()
        net.eval()
        weight_count = _calc_width(net)
        print("m={}, {}".format(model.__name__, weight_count))
        assert (model != fractalnet_cifar10 or weight_count == 33724618)
        assert (model != fractalnet_cifar100 or weight_count == 33770788)

        x = torch.randn(1, 3, 32, 32)
        y = net(x)
        y.sum().backward()
        assert (tuple(y.size()) == (1, num_classes))


if __name__ == "__main__":
    _test()

# Dataset

In [ ]:
class UBCDataset(Dataset):
    def __init__(self, df, transforms=None):
        # 📊 Initialize the dataset with DataFrame, file names, labels, and transformations
        self.df = df
        self.file_names = df['file_path'].values
        self.labels = df['label'].values
        self.transforms = transforms
        self.sxs = df["sx"].values
        self.exs = df["ex"].values
        self.sys = df["sy"].values
        self.eys = df["ey"].values

    def __len__(self):
        # 🔄 Return the length of the dataset
        return len(self.df)

    def __getitem__(self, index):
        # 🔍 Get an item from the dataset based on the index

        # 📄 Get image path, cropping boundaries, and label
        img_path = self.file_names[index]
        sx, ex, sy, ey = self.sxs[index], self.exs[index], self.sys[index], self.eys[index]

        # 🖼️ Read and convert the image to RGB
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # 🔄 Crop the image based on cropping boundaries
        img = img[sy:ey, sx:ex, :]

        # 🔍 Get the label
        label = self.labels[index]

        # 🔄 Apply transformations if specified
        if self.transforms:
            img = self.transforms(image=img)["image"]

        # 🔍 Return a dictionary containing the image and label as torch tensors
        return {
            'image': img,
            'label': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
model = CIFARFractalNet(channels=7,
                 num_columns=5,
                 dropout_probs=0.34,
                 loc_drop_prob=0.3,
                 glob_drop_ratio=0.24,
                 in_channels=3,
                 in_size=(40000, 20000),
                 num_classes=6)

with torch.no_grad():
    bar = tqdm(enumerate(test_loader), total=len(test_loader))
    for step, data in bar:
        # 🔄 Move the input images to the specified device
        images = data['image'].to(CONFIG["device"], dtype=torch.float)

        # 🔍 Forward pass through the models and combine the outputs
        output = model(images)

Epoch 1/30
205/205 [==============================] - 40s 121ms/step - loss: 1.4622 - acc: 0.4448 - auc: 0.7062 - val_loss: 0.8332 - val_acc: 0.6034 - val_auc: 0.8729
Epoch 2/30
205/205 [==============================] - 21s 101ms/step - loss: 0.9421 - acc: 0.5810 - auc: 0.8457 - val_loss: 0.6924 - val_acc: 0.6724 - val_auc: 0.9099
Epoch 3/30
205/205 [==============================] - 22s 108ms/step - loss: 0.7865 - acc: 0.6376 - auc: 0.8864 - val_loss: 0.6511 - val_acc: 0.6840 - val_auc: 0.9182
Epoch 4/30
205/205 [==============================] - 21s 104ms/step - loss: 0.7334 - acc: 0.6664 - auc: 0.9016 - val_loss: 0.6234 - val_acc: 0.7041 - val_auc: 0.9266
Epoch 5/30
205/205 [==============================] - 22s 109ms/step - loss: 0.6941 - acc: 0.6850 - auc: 0.9115 - val_loss: 0.5871 - val_acc: 0.7181 - val_auc: 0.9333
Epoch 6/30
205/205 [==============================] - 23s 110ms/step - loss: 0.6513 - acc: 0.7061 - auc: 0.9224 - val_loss: 0.5778 - val_acc: 0.7224 - val_auc: 0.9360
Epoch 7/30
205/205 [==============================] - 22s 108ms/step - loss: 0.6225 - acc: 0.7305 - auc: 0.9316 - val_loss: 0.5430 - val_acc: 0.7364 - val_auc: 0.9426
Epoch 8/30
205/205 [==============================] - 23s 111ms/step - loss: 0.5784 - acc: 0.7453 - auc: 0.9395 - val_loss: 0.5263 - val_acc: 0.7444 - val_auc: 0.9470
Epoch 9/30
205/205 [==============================] - 23s 112ms/step - loss: 0.5700 - acc: 0.7564 - auc: 0.9428 - val_loss: 0.5156 - val_acc: 0.7608 - val_auc: 0.9503
Epoch 10/30
205/205 [==============================] - 22s 108ms/step - loss: 0.5313 - acc: 0.7748 - auc: 0.9495 - val_loss: 0.4930 - val_acc: 0.7785 - val_auc: 0.9548
Epoch 11/30
205/205 [==============================] - 22s 107ms/step - loss: 0.5311 - acc: 0.7798 - auc: 0.9501 - val_loss: 0.4796 - val_acc: 0.7889 - val_auc: 0.9576
Epoch 12/30
205/205 [==============================] - 23s 111ms/step - loss: 0.4747 - acc: 0.8086 - auc: 0.9599 - val_loss: 0.4781 - val_acc: 0.7871 - val_auc: 0.9588
Epoch 13/30
205/205 [==============================] - 22s 108ms/step - loss: 0.4821 - acc: 0.8019 - auc: 0.9585 - val_loss: 0.4633 - val_acc: 0.7962 - val_auc: 0.9607
Epoch 14/30
205/205 [==============================] - 23s 111ms/step - loss: 0.4534 - acc: 0.8218 - auc: 0.9634 - val_loss: 0.4974 - val_acc: 0.7913 - val_auc: 0.9572
Epoch 15/30
205/205 [==============================] - 22s 108ms/step - loss: 0.4438 - acc: 0.8247 - auc: 0.9647 - val_loss: 0.4296 - val_acc: 0.8139 - val_auc: 0.9666
Epoch 16/30
205/205 [==============================] - 23s 112ms/step - loss: 0.4145 - acc: 0.8424 - auc: 0.9690 - val_loss: 0.4588 - val_acc: 0.8127 - val_auc: 0.9628
Epoch 17/30
205/205 [==============================] - 23s 112ms/step - loss: 0.4139 - acc: 0.8392 - auc: 0.9692 - val_loss: 0.4225 - val_acc: 0.8310 - val_auc: 0.9689
Epoch 18/30
205/205 [==============================] - 23s 111ms/step - loss: 0.4079 - acc: 0.8492 - auc: 0.9706 - val_loss: 0.4261 - val_acc: 0.8359 - val_auc: 0.9692
Epoch 19/30
205/205 [==============================] - 23s 111ms/step - loss: 0.3893 - acc: 0.8491 - auc: 0.9727 - val_loss: 0.3999 - val_acc: 0.8383 - val_auc: 0.9713
Epoch 20/30
205/205 [==============================] - 22s 108ms/step - loss: 0.3775 - acc: 0.8564 - auc: 0.9744 - val_loss: 0.4147 - val_acc: 0.8353 - val_auc: 0.9706
Epoch 21/30
205/205 [==============================] - 22s 107ms/step - loss: 0.3489 - acc: 0.8688 - auc: 0.9777 - val_loss: 0.3992 - val_acc: 0.8395 - val_auc: 0.9723
Epoch 22/30
205/205 [==============================] - 23s 112ms/step - loss: 0.3699 - acc: 0.8669 - auc: 0.9752 - val_loss: 0.4094 - val_acc: 0.8426 - val_auc: 0.9714
Epoch 23/30
205/205 [==============================] - 22s 108ms/step - loss: 0.3189 - acc: 0.8782 - auc: 0.9811 - val_loss: 0.3823 - val_acc: 0.8505 - val_auc: 0.9753
Epoch 24/30
205/205 [==============================] - 24s 116ms/step - loss: 0.3162 - acc: 0.8840 - auc: 0.9812 - val_loss: 0.4102 - val_acc: 0.8456 - val_auc: 0.9718
Epoch 25/30
205/205 [==============================] - 23s 111ms/step - loss: 0.3280 - acc: 0.8840 - auc: 0.9802 - val_loss: 0.4007 - val_acc: 0.8438 - val_auc: 0.9735
Epoch 26/30
205/205 [==============================] - 22s 108ms/step - loss: 0.3079 - acc: 0.8865 - auc: 0.9824 - val_loss: 0.3691 - val_acc: 0.8572 - val_auc: 0.9761
Epoch 27/30
205/205 [==============================] - 22s 108ms/step - loss: 0.2893 - acc: 0.8962 - auc: 0.9843 - val_loss: 0.4023 - val_acc: 0.8542 - val_auc: 0.9737
Epoch 28/30
205/205 [==============================] - 23s 111ms/step - loss: 0.2991 - acc: 0.8944 - auc: 0.9829 - val_loss: 0.3747 - val_acc: 0.8548 - val_auc: 0.9760
Epoch 29/30
205/205 [==============================] - 23s 112ms/step - loss: 0.2814 - acc: 0.8970 - auc: 0.9848 - val_loss: 0.3763 - val_acc: 0.8646 - val_auc: 0.9765
Epoch 30/30
205/205 [==============================] - 23s 112ms/step - loss: 0.2810 - acc: 0.9060 - auc: 0.9846 - val_loss: 0.3822 - val_acc: 0.8621 - val_auc: 0.9761



### Automatically apply Gaussian blur and downsample images at different scales (currently 2x, 3x, and 4x) using OpenCV's implementation of bicubic interpolation. This utility can be especially useful in generating training and testing data sets for super-resolution tasks.

In [ ]:
import os
import argparse
import cv2


# # Parse args
# parser = argparse.ArgumentParser(description='Downsize images at 2x, 3x, and 4x\
#     using bicubic interpolation.')
# parser.add_argument("hr_img_dir", help="path to high resolution image dir")
# parser.add_argument("lr_img_dir", help="path to desired output dir for\
#     downsampled images")
# parser.add_argument("-k", "--keepdims", help="keep original image dimensions in\
#     downsampled images", action="store_true")
# args = parser.parse_args()

hr_image_dir = "/kaggle/input/UBC-OCEAN/train_images"
lr_image_dir = "/kaggle/working/lr"

# Create LR image dirs
os.makedirs(lr_image_dir + "/2x", exist_ok=True)
os.makedirs(lr_image_dir + "/3x", exist_ok=True)
os.makedirs(lr_image_dir + "/4x", exist_ok=True)

supported_img_formats = (".bmp", ".dib", ".jpeg", ".jpg", ".jpe", ".jp2",
                         ".png", ".pbm", ".pgm", ".ppm", ".sr", ".ras", ".tif",
                         ".tiff")

# Downsample HR images
for filename in os.listdir(hr_image_dir):
    if not filename.endswith(supported_img_formats):
        continue

    # Read HR image
    hr_img = cv2.imread(os.path.join(hr_image_dir, filename))
    hr_img_dims = (hr_img.shape[1], hr_img.shape[0])

    # Blur with Gaussian kernel of width sigma=1
    hr_img = cv2.GaussianBlur(hr_img, (0, 0), 1, 1)

    # Downsample image 2x
    lr_img_2x = cv2.resize(hr_img, (0, 0), fx=0.5, fy=0.5,
                           interpolation=cv2.INTER_CUBIC)
    lr_img_2x = cv2.resize(lr_img_2x, hr_img_dims,
                               interpolation=cv2.INTER_CUBIC)
    cv2.imwrite(os.path.join(lr_image_dir + "/2x", filename), lr_img_2x)

    # Downsample image 3x
    lr_img_3x = cv2.resize(hr_img, (0, 0), fx=(1 / 3), fy=(1 / 3),
                           interpolation=cv2.INTER_CUBIC)
    lr_img_3x = cv2.resize(lr_img_3x, hr_img_dims,
                               interpolation=cv2.INTER_CUBIC)
    cv2.imwrite(os.path.join(lr_image_dir + "/3x", filename), lr_img_3x)

    # Downsample image 4x
    lr_img_4x = cv2.resize(hr_img, (0, 0), fx=0.25, fy=0.25,
                           interpolation=cv2.INTER_CUBIC)
    lr_img_4x = cv2.resize(lr_img_4x, hr_img_dims,
                               interpolation=cv2.INTER_CUBIC)
    cv2.imwrite(os.path.join(lr_image_dir + "/4x", filename), lr_img_4x)